In [ ]:
#%pip install numpy
#%pip install pandas
#%pip install ipykernel
#%pip install plotly
#%pip install nbformat 

In [ ]:
import os
import numpy as np
import pandas as pd
import json

#raw data folder name
_rawData = '/rawData/'
#labeled dataset folder name 
_labeledData = '/labeledData/'

# Path to data
dataPath = os.getcwd()+'/dataset/'
rawDataPath = dataPath + _rawData
labeledDataPath = dataPath + _labeledData
saveDatesetPath = dataPath + '/output/'


# Extract and syncronize data 
it extact data from each trial and save it to 
    /dataset/labeledData/

In [ ]:
import numpy as np
import pandas as pd
import os 

class MakeFolderDataset:
    def __init__(self, folder_path:str) -> None:
        self.path = folder_path
        self.save_path = folder_path.replace(_rawData, _labeledData )
        #os.makedirs(self.save_path)
        self.num_lines_per_message = 130
        self.df = pd.DataFrame()
        self.df_dataset = pd.DataFrame()
        self.true_label = pd.DataFrame()
        self.tau = ['tau_J0','tau_J1', 'tau_J2', 'tau_J3', 'tau_J4', 'tau_J5', 'tau_J6']
        self.tau_d = ['tau_J_d0','tau_J_d1', 'tau_J_d2', 'tau_J_d3', 'tau_J_d4', 'tau_J_d5', 'tau_J_d6']
        self.tau_ext =['tau_ext0','tau_ext1','tau_ext2','tau_ext3','tau_ext4','tau_ext5','tau_ext6']

        self.q = ['q0','q1','q2','q3','q4','q5','q6']
        self.q_d = ['q_d0','q_d1','q_d2','q_d3','q_d4','q_d5','q_d6']

        self.dq = ['dq0','dq1','dq2','dq3','dq4','dq5','dq6']
        self.dq_d = ['dq_d0','dq_d1','dq_d2','dq_d3','dq_d4','dq_d5','dq_d6']


        self.e = ['e0','e1','e2','e3','e4','e5','e6']
        self.de = ['de0','de1','de2','de3','de4','de5','de6']
        self.etau = ['etau_J0','etau_J1', 'etau_J2', 'etau_J3', 'etau_J4', 'etau_J5', 'etau_J6']
    
    def _extract_array(self, data_dict:dict, data_frame:str, header:list,  n:int):
            _, y = data_frame[n].split(':')
            y = y.replace('[','')
            y = y.replace(']','')
            y = y.replace('\n','')
            y = y.split(',')
            for i, h in enumerate(header):
                data_dict[h].append(float(y[i]))

    def extract_robot_data(self):
        # it extracts robot data from all_data.txt
        f = open(self.path + 'all_data.txt', 'r')
        lines = f.readlines()

        keywords = ['time'] + self.tau + self.tau_d + self.tau_ext + self.q + self.q_d + self.dq + self.dq_d 
        data_dict = dict.fromkeys(keywords)
        for i in keywords:
            data_dict[i]=[0]
        
        for i in range(int(len(lines)/self.num_lines_per_message)):
            data_frame = lines[i*self.num_lines_per_message:(i+1)*self.num_lines_per_message]
            
            # extract exact time from file sec + nsec
            _, y = data_frame[3].split(':')
            time_ = int(y)-int(int(y)/1000000)*1000000

            _, y = data_frame[4].split(':')
            time_ = time_+int(y)/np.power(10,9)

            data_dict['time'].append(time_)
            
            self._extract_array(data_dict,data_frame,self.tau, 25)
            self._extract_array(data_dict,data_frame,self.tau_d, 26)
            self._extract_array(data_dict,data_frame, self.tau_ext, 37)
            
            self._extract_array(data_dict,data_frame,self.q, 28)
            
            self._extract_array(data_dict,data_frame, self.q_d, 29)
            self._extract_array(data_dict,data_frame, self.dq, 30)
            self._extract_array(data_dict,data_frame, self.dq_d, 31)
        
       
        self.df = pd.DataFrame.from_dict(data_dict)
        self.df = self.df.drop(index=0).reset_index()
        for i in range(len(self.e)):
            self.df[self.e[i]] = self.df[self.q_d[i]]-self.df[self.q[i]]
            self.df[self.de[i]] = self.df[self.dq_d[i]]-self.df[self.dq[i]]
            self.df[self.etau[i]] = self.df[self.tau_d[i]]-self.df[self.tau[i]]

    def get_labels(self):
        # it syncronize true labeled (contact- noncontact) data with robot data
        true_label = pd.read_csv(self.path+'true_label.csv')
        true_label['time'] = true_label['time_sec']+true_label['time_nsec']-self.df['time'][0]
        time_dev = true_label['time'].diff()
        contact_events_index = np.append([0], true_label['time'][time_dev>0.05].index.values)
        contact_events_index = np.append(contact_events_index,  true_label['time'].shape[0]-1)

        self.df['time'] = self.df['time'] - self.df['time'][0]
        contact_count = 0
        self.df['label']=0

        for i in range(self.df['time'].shape[0]):
            if (self.df['time'][i]-true_label['time'][contact_events_index[contact_count]]) > 0:
                #print(i ,',', contact_events_index[contact_count], ',',self.df['time'][i], '   ', true_label['time'][contact_events_index[contact_count]] )
                contact_count += 1
                if contact_count == len(contact_events_index):
                    break
                for j in range(i, self.df['time'].shape[0]):
                    self.df.loc[j, 'label'] = 1
                    #print(j)
                    if (self.df['time'][j] - true_label['time'][contact_events_index[contact_count]-1]) > 0:
                        #print(j ,',', contact_events_index[contact_count]-1, ',', self.df['time'][j], '   ', true_label['time'][contact_events_index[contact_count]-1] )
                        #print('----------------------------------------')
                        i = j
                        break

        self.df.to_csv(self.save_path + 'labeled_data.csv', index=False)

    def get_labels_All(self):
        self.true_label = pd.read_csv(self.path+'true_label.csv')
        self.true_label['time'] = self.true_label['time_sec']+self.true_label['time_nsec']-self.df['time'][0]
        self.df['time'] = self.df['time'] - self.df['time'][0]
        contact_count = 0
        self.df['label']= 0
      
        
        self.true_label['time_dev'] = self.true_label['time'].diff()
        
        # Find indices where 'time_dev' is greater than 0.05
        indices = self.true_label[self.true_label['time_dev'] > 0.05].index

        # Iterate through indices in reverse order to avoid index shifting issues
        for i in indices[::-1]:
     
            # Create a new row with the same values except for DATA0 which is set to 0
            new_row = self.true_label.iloc[i].to_frame().T
            new_row['DATA0'] = 0
            
            # Concatenate the DataFrame slices properly to avoid duplication
            self.true_label = pd.concat([self.true_label.iloc[:i], new_row, self.true_label.iloc[i:]])
            

        # Reset index after concatenation
        self.true_label = self.true_label.reset_index(drop=True)

        # Sort the DataFrame by the 'time' column to maintain order
        self.true_label = self.true_label.sort_values(by='time')
        
    def make_sequence(self):
        #window_time = 140ms
        seq_num = 28
        gap = 4
        selected_features= self.e + self.tau

        dataset = pd.DataFrame(np.ones((int((self.df.shape[0]-seq_num)/gap), seq_num*len(selected_features)+1))*2 )
        index = 0
        state = False
        last_contact_indexes = self.df.loc[self.df['label']==1,'index'].values
        last_contact_indexes = last_contact_indexes[last_contact_indexes.shape[0]-1]

        for i in range(0, last_contact_indexes, gap):
            if state: 
                window = self.df[selected_features][i:i+seq_num]
                dataset.iloc[index,0] = self.df['label'][i+seq_num]
                dataset.iloc[index, 1:len(dataset.columns)] = np.hstack(window.to_numpy())
                index += 1
            else:
                if self.df['label'][i+seq_num] == 1:
                    state = 1
        self.dataset = dataset.drop(index=dataset.loc[dataset[0]==2,0].index)

        name = self.path.split('/')[len(self.path.split('/'))-2]+'.csv'
        self.dataset.to_csv(self.save_path+name, index=False)
        return self.dataset
    
    def split_data(self, train_split_rate = 0.75):
        msk = np.random.rand(len(self.dataset)) < train_split_rate
        train = self.dataset.loc[msk, :]
        test = self.dataset.loc[~msk, :]
        name = self.path.split('/')[len(self.path.split('/'))-2]+'_train.csv'
        train.to_csv(self.save_path+name, index=False)

        name = self.path.split('/')[len(self.path.split('/'))-2]+'_test.csv'
        test.to_csv(self.save_path+name, index=False)



In [ ]:
#instance = MakeFolderDataset(r"C:\Users\marco\OneDrive - Universität Zürich UZH\Master_UZH\master_project\dataset\rawData\configuration2\hard\")
#instance = MakeFolderDataset(r"C:\Users\marco\OneDrive - Universität Zürich UZH\Master_UZH\master_project\dataset\rawData\configuration2\hard\"")
instance1 = MakeFolderDataset("C:\\Users\\juhe9\\repos\\MasterProject\\humanObjectDetection\\dataLabeling\\dataset\\rawData\\configuration1\\multipleHardwall\\")
instance1.extract_robot_data()
instance1.get_labels_All()

instance2 = MakeFolderDataset("C:\\Users\\juhe9\\repos\\MasterProject\\humanObjectDetection\\dataLabeling\\dataset\\rawData\\configuration1\\multiplesoftwall\\")
instance2.extract_robot_data()
instance2.get_labels_All()

#instance.true_label.head(100)
(instance1.true_label["DATA0"] == 1).sum()
(instance2.true_label["DATA0"] == 1).sum()

instances =[instance1, instance2]

# SIEHE get_labels_All --> DATA0 bei diff > 0.05 row inserted mit DATA0 = 0

In [ ]:

import plotly.graph_objs as go
from plotly.offline import iplot

target = ['etau_J0','etau_J1', 'etau_J2', 'etau_J3', 'etau_J4', 'etau_J5', 'etau_J6']
#target = ['e0','e1','e2','e3','e4','e5','e6']
#target = ['de0','de1','de2','de3','de4','de5','de6']
for i in target:
    for inst in instances:
        A = inst.df[i].max()-inst.df[i].min()
        # label gets scaled otherwise measure and label are not visible properly on plot
        inst.true_label['label_scaled']=inst.true_label['DATA0']*A + inst.df[i].min()     #instance.df[i][0] -A/2
        # Assuming df1 and df2 are your two DataFrames with x and y data
        trace1 = go.Scatter(x=inst.true_label['time'], y=inst.true_label['label_scaled'], name='true label')
        trace2 = go.Scatter(x=inst.df['time'], y=inst.df[i], mode='lines', name='robot data')
        data = [trace1, trace2]
        layout = go.Layout(title=f'{i} (instance {os.path.basename(os.path.normpath(inst.path))})',
                        xaxis=dict(title='time(sec)'),
                        yaxis=dict(title='Y-axis'))
        fig = go.Figure(data=data, layout=layout)
        iplot(fig)

In [ ]:

for i in os.listdir(rawDataPath):
    if len(i.split('.'))==1:
        submain_path = rawDataPath+i+'/'
        print(submain_path)
        for j in os.listdir(submain_path):
            if len(j.split('.'))==1:
                instance = MakeFolderDataset(submain_path+j+'/')
                instance.extract_robot_data()
                instance.get_labels()
                
instance.df.head()

In [ ]:
(instance.df["label"] == 0).sum()

# plot data

In [ ]:
from plotly.offline import iplot, init_notebook_mode
import cufflinks
# Using plotly + cufflinks in offline mode
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [ ]:
target = ['etau_J0','etau_J1', 'etau_J2', 'etau_J3', 'etau_J4', 'etau_J5', 'etau_J6']

for i in target:
    A = instance.df[i].max()-instance.df[i].min()
    # label gets scaled otherwise measure and label are not visible properly on plot
    instance.df['label_scaled']=instance.df['label']*A + instance.df[i][0] -A/2
    instance.df.iplot(x='time', y= [i, 'label_scaled'], xTitle='time (sec)', yTitle=i)
    
    

joint position errors gives good plots --> self.e

joint velocity errors bad marker not really visible on plot what (wie eine wolke haha)  --> self.de

joint torque errors similar to joint position --> self.etau

too many arduino labels, increase threshold of touch 
how fast is our sampling rate per second? maybe to low 

# Make Dataset


## Features

In [ ]:
tau = ['tau_J0','tau_J1', 'tau_J2', 'tau_J3', 'tau_J4', 'tau_J5', 'tau_J6']
tau_d = ['tau_J_d0','tau_J_d1', 'tau_J_d2', 'tau_J_d3', 'tau_J_d4', 'tau_J_d5', 'tau_J_d6']
tau_ext =['tau_ext0','tau_ext1','tau_ext2','tau_ext3','tau_ext4','tau_ext5','tau_ext6']

q = ['q0','q1','q2','q3','q4','q5','q6']
q_d = ['q_d0','q_d1','q_d2','q_d3','q_d4','q_d5','q_d6']

dq = ['dq0','dq1','dq2','dq3','dq4','dq5','dq6']
dq_d = ['dq_d0','dq_d1','dq_d2','dq_d3','dq_d4','dq_d5','dq_d6']


e = ['e0','e1','e2','e3','e4','e5','e6']
de = ['de0','de1','de2','de3','de4','de5','de6']
etau = ['etau_J0','etau_J1', 'etau_J2', 'etau_J3', 'etau_J4', 'etau_J5', 'etau_J6']

## required functions

In [ ]:
def split_data(dataset, train_split_rate = 0.75):
    msk = np.random.rand(len(dataset)) < train_split_rate
    train = dataset.loc[msk, :]
    test = dataset.loc[~msk, :]
    return train, test

def make_sequence(df, selected_features, seq_num = 28, gap = 4):
    dataset = pd.DataFrame(np.ones((int((df.shape[0]-seq_num)/gap), seq_num*len(selected_features)+1))*2 )
    index = 0
    state = False
    last_contact_indexes = df.loc[df['label']==1,'index'].values
    last_contact_indexes = last_contact_indexes[last_contact_indexes.shape[0]-1]

    for i in range(0, last_contact_indexes, gap):
        if state: 
            window =df[selected_features][i:i+seq_num]
            dataset.iloc[index,0] = df['label'][i+seq_num]
            dataset.iloc[index, 1:len(dataset.columns)] = np.hstack(window.to_numpy())
            index += 1
        else:
            if df['label'][i+seq_num] == 1:
                state = 1
    dataset = dataset.drop(index=dataset.loc[dataset[0]==2,0].index)
    return dataset

def make_sequence_full_time(df, selected_features, seq_num = 28, gap = 4):
    dataset = pd.DataFrame(np.ones((int((df.shape[0]-seq_num)), seq_num*len(selected_features)+1))*2 )
    index = 0
    last_contact_indexes = df.loc[df['label']==1,'index'].values
    last_contact_indexes = last_contact_indexes[last_contact_indexes.shape[0]-1]

    for i in range(0, df.shape[0]-seq_num-1, gap):

        window =df[selected_features][i:i+seq_num]
        dataset.iloc[index,0] = df['label'][i+seq_num]
        dataset.iloc[index, 1:len(dataset.columns)] = np.hstack(window.to_numpy())
        index += 1

    dataset = dataset.drop(index=dataset.loc[dataset[0]==2,0].index)
    return dataset

## make a dataset with a set of features


In [ ]:
os.makedirs(saveDatesetPath)
train_split_rate = 0.7

selected_features = tau + tau_ext + e + de
seq_num = 28

dict_label = {'a': 7, 'b':6, 'c':5, 'd':4, 'e':3, 'f':2, 'g':1}
columns = range(seq_num*len(selected_features)+1)

df_master_master_train = pd.DataFrame(columns=columns)
df_master_master_test = pd.DataFrame(columns=columns)


    
for i in os.listdir(labeledDataPath):
    if len(i.split('.'))==1:

        df_master_train = pd.DataFrame(columns=columns)
        df_master_test = pd.DataFrame(columns=columns)

        submain_path = labeledDataPath+i+'/'
        print(submain_path)
        for tag_name in os.listdir(submain_path):
            if len(tag_name.split('.'))==1:
                file_path = submain_path+tag_name+'/labeled_data.csv'
                df = pd.read_csv(file_path)
                df = make_sequence_full_time(df, selected_features,seq_num)
                #labeling data
                df[0] = df[0]*dict_label[tag_name[0]]
                train, test = split_data(df, train_split_rate)
                df_master_train = df_master_train.append(train, ignore_index=True)
                df_master_test = df_master_test.append(test, ignore_index=True)
                
        df_master_train.to_pickle(saveDatesetPath+i+'_train.pkl')
        df_master_test.to_pickle(saveDatesetPath+i+'_test.pkl')
        
    df_master_master_train = df_master_master_train.append(df_master_train, ignore_index=True)
    df_master_master_test = df_master_master_test.append(df_master_test, ignore_index=True)

df_master_master_train.to_pickle(saveDatesetPath+'dataset_train.pkl')
df_master_master_test.to_pickle(saveDatesetPath+'dataset_test.pkl')


# # Print Dataset Information

In [ ]:
print('number of all samples: ', df_master_master_train.shape[0]+df_master_master_test.shape[0])
print('number of noncontact samples: ', df_master_master_train[0][df_master_master_train[0]==0].shape[0] +  df_master_master_test[0][df_master_master_test[0]==0].shape[0] )
print('number of contact samples: ', df_master_master_train[0][df_master_master_train[0]!=0].shape[0] +  df_master_master_test[0][df_master_master_test[0]!=0].shape[0] )
print('Link1 samples: ', df_master_master_train[0][df_master_master_train[0]==1].shape[0] +  df_master_master_test[0][df_master_master_test[0]==1].shape[0] )
print('Link2 samples: ', df_master_master_train[0][df_master_master_train[0]==2].shape[0] +  df_master_master_test[0][df_master_master_test[0]==2].shape[0] )
print('Link3 samples: ', df_master_master_train[0][df_master_master_train[0]==3].shape[0] +  df_master_master_test[0][df_master_master_test[0]==3].shape[0] )
print('Link4 samples: ', df_master_master_train[0][df_master_master_train[0]==4].shape[0] +  df_master_master_test[0][df_master_master_test[0]==4].shape[0] )
print('Link5 samples: ', df_master_master_train[0][df_master_master_train[0]==5].shape[0] +  df_master_master_test[0][df_master_master_test[0]==5].shape[0] )
print('Link6 samples: ', df_master_master_train[0][df_master_master_train[0]==6].shape[0] +  df_master_master_test[0][df_master_master_test[0]==6].shape[0] )
print('Link7 samples: ', df_master_master_train[0][df_master_master_train[0]==7].shape[0] +  df_master_master_test[0][df_master_master_test[0]==7].shape[0] )